In [ ]:
import base
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rushd as rd
import scipy as sp
import seaborn as sns

# enables concurrent editing of base.py
from importlib import reload
reload(base)

### Load data

Circuit tuning data (`data`)

In [ ]:
base_path = rd.datadir/'instruments'/'data'/'attune'
metadata_path = rd.datadir/'projects'/'miR-iFFL'/'plasmids'
data, quantiles, stats, metadata = base.load_data(base_path, metadata_path, 'tuning', 'designs')

In [ ]:
# Add shades to teal color palette
#  light = EFS, miR-FF4, miRE-FF5, TSx2
#  dark = TSx4
metadata.loc[metadata['promoter']=='EFS', 'color'] = metadata.loc[metadata['promoter']=='EFS', 'color'].apply(base.get_dark_color)
metadata.loc[metadata['miR']=='miR.FF4', 'color'] = metadata.loc[metadata['miR']=='miR.FF4', 'color'].apply(base.get_dark_color)
metadata.loc[metadata['miR']=='miRE.FF5', 'color'] = metadata.loc[metadata['miR']=='miRE.FF5', 'color'].apply(base.get_light_color)
metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==2)), 'color'] = metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==2)), 'color'].apply(base.get_light_color)
metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'] = metadata.loc[((metadata['group']=='controller') & (metadata['ts_num']==4)), 'color'].apply(base.get_dark_color)

# Create dicts to specify colors/markers
metadata_dict = metadata.set_index('construct').to_dict('dict')
main_palette = metadata_dict['color']
main_markers = metadata_dict['markers']

greys = matplotlib.colors.ListedColormap(matplotlib.colormaps['Greys'](np.linspace(0.2,0.7,256)))

Load modeling parameter sweeps

In [ ]:
simulation_path = rd.rootdir/'output'
sim_data, slopes = base.load_modeling(simulation_path, 'param_sweeps')

### Set up figure

In [ ]:
# Set plotting context
sns.set_style('ticks')
sns.set_context('paper', font_scale=1.0, rc=base.rc_context)
plt.rcParams.update(base.rc_params)
scatter_kwargs = base.scatter_kwargs

# Create the overall figure, gridspec, and add subfigure labels
first_col = 2.75
fig = plt.figure(figsize=(base.figure_width['full'], 7.875))
fig_gridspec = matplotlib.gridspec.GridSpec(5, 3, figure=fig, wspace=0.4, hspace=0.4, 
                                            height_ratios=[1.5+1/12]*3+[1.625,1.5], 
                                            width_ratios=[first_col,(base.figure_width['full']-first_col)/2,(base.figure_width['full']-first_col)/2])

subfigures = {
    'A': fig.add_subfigure(fig_gridspec[:3,0]),
    'B': fig.add_subfigure(fig_gridspec[0,1:]),
    'C': fig.add_subfigure(fig_gridspec[1,1:]),
    'D': fig.add_subfigure(fig_gridspec[2,1:]),
    'E': fig.add_subfigure(fig_gridspec[3,0]),
    'E2': fig.add_subfigure(fig_gridspec[3,1:]),
    'G': fig.add_subfigure(fig_gridspec[4,0]),
    'H': fig.add_subfigure(fig_gridspec[4,1:]),
}

# Add subpanel labels
for label, subfig in subfigures.items():
    if '2' in label: continue
    subfig.add_artist(matplotlib.text.Text(x=0, y=1, text=f'{label}', fontsize=base.font_sizes['subpanel_label'], 
                                           fontweight='bold', verticalalignment='top',transform=subfig.transSubfigure))

# Save to output folder
output_path = rd.rootdir/'output'/'fig_model'/'fig_model.pdf'
fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['B']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: basic params
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='Rtot']
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='unreg', color=base.colors['gray'], ax=ax,
             errorbar=None,)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], title='Initial parameters', ylabel='protein (#)')
ax.axvspan(55,100, color=base.get_light_color(base.colors['gray']), alpha=0.2)
setpoint = plot_df.loc[(plot_df['param_val_norm']==1) & (plot_df['copy_num']==50), 'protein'].values[0]
ax.axhline(setpoint, color=base.colors['teal'], ls=':', xmax=100, zorder=1)

# model: Rtot sweep
ax = axes[1]
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,12e3),
       yticklabels=['']*len(yticks), ylabel='')
ax.set_title('Total RISC, $R_{tot}$', color=base.colors['green'])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['C']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: k_mRNA_bind vs protein at set copy number
ax = axes[0]
plot_df = slopes[(slopes['param']=='α_im')]
sns.scatterplot(data=plot_df, x='param_val_norm', y='slope_norm', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False, s=2.25**2*3.14, linewidth=0.5, edgecolor='white')
sns.scatterplot(data=plot_df[plot_df['param_val_norm']==1], x='param_val_norm', y='slope_norm', color='black', 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False, s=2.25**2*3.14, linewidth=0.5, edgecolor='white')
ax.set(xlabel='Norm. $α_{RNA}$', ylabel='slope')
ax.set_title('Transcription, $α_{RNA}$', color=base.colors['blue'])
threshold = plot_df['param_val_norm'].unique()[2] + (plot_df['param_val_norm'].unique()[3] - plot_df['param_val_norm'].unique()[2])/2
ax.axvspan(threshold,ax.get_xlim()[1], color=base.get_light_color(base.colors['gray']), alpha=0.2)

# data: miRE-FF4, EF1a vs EFS
ax = axes[1]
promoter_order = ['EFS','EF1a']
plot_df2 = stats[(((stats['miR']=='miRE.FF4') & (stats['group']=='controller') & (stats['design']==1) & (stats['ts'].isin(['FF4x1','FF6x1']))) &
                 stats['promoter'].isin(promoter_order))]
plot_df2.sort_values(['group','ts_kind'], inplace=True)
plot_df2.sort_values('promoter', ascending=False, inplace=True)
display(plot_df2['construct'].unique())

for construct, group in plot_df2.groupby('construct', sort=False):
       sns.pointplot(data=group, x='promoter', y='slope', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs, native_scale=True)
ax.set(title='Experimental data', xlabel='', ylim=(0,1.2), ylabel='slope', 
       yticks=[0,0.25,0.5,0.75,1], yticklabels=['0.0','','0.5','','1.0'], 
       xticklabels=[l.get_text().replace('a','α') for l in ax.get_xticklabels()])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['D']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.35, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.5))

# model: r_drosha sweep
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='r_drosha']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val_norm', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)

ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('Drosha processing, $r_{drosha}$', color=base.colors['purple'])

# data: miRE-FF4 vs miR-FF4
ax = axes[1]
biorep = 1
plot_df = quantiles[((quantiles['miR'].isin(['miRE.FF4','miR.FF4'])) & (quantiles['group']=='controller') & (quantiles['design']==1) & (quantiles['ts'].isin(['FF4x1','FF5x1','FF6x1']))) &
                       (quantiles['promoter']=='EF1a') & (quantiles['biorep']==biorep)]
display(plot_df['construct'].unique())

sns.lineplot(data=plot_df, x='bin_marker_quantiles_median', y='output', hue='construct', palette=main_palette, 
             legend=False, dashes=False, style='construct', markers=main_markers, ax=ax, markersize=4, markeredgewidth=0.5,
             estimator=sp.stats.gmean, errorbar=lambda x: (sp.stats.gmean(x) / sp.stats.gstd(x), sp.stats.gmean(x) * sp.stats.gstd(x)))
ax.set(xscale='log', yscale='log', xlabel='marker', xlim=(2e2,6e4), ylim=(2e1,1e5), yticks=(1e2,1e3,1e4,1e5), title='Experimental data')
ax.minorticks_off()
marker_baseline = sp.stats.gmean(quantiles.loc[(quantiles['group']=='marker') & (quantiles['biorep']==biorep), 'output'])
ax.axhline(marker_baseline, color='black', ls=':')

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.45, top=0.4, right=0.35)
axes = subfig.subplots(1,2, gridspec_kw=dict(wspace=0.2, width_ratios=(1,0.75)))

# model: k_mRNA_bind sweep
ax = axes[0]
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='k_mRNA_bind']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('RISC-mRNA binding,\n$k_{mRNA,bind}$', color=base.colors['red'])

# add annotations
param_val_subset = [p for i,p in enumerate(plot_df['param_val_norm'].unique()) if i%2==1]
plot_df = sim_data[(sim_data['param']=='k_mRNA_bind') & (sim_data['param_val_norm'].isin(param_val_subset)) & (plot_df['copy_num'].isin([25,75]))]
reds = sns.light_palette(color=base.colors['red'], n_colors=7)[1:6]
greys2 = greys.colors
greys2[128] = (0,0,0,1)
greys_subset = [l.tolist() for i,l in enumerate(greys2) if i%64==0]
greys_subset.append(greys2[-1])
ax.axvline(x=25, color=reds[1], ymax=(11e3)/(12e3))
ax.axvline(x=75, color=reds[4], ymax=(11e3)/(12e3))
sns.scatterplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys_subset,
                ax=ax, s=2.25**2*3.14, linewidth=0.5, edgecolor='white', legend=False, zorder=100)

# model: k_mRNA_bind vs protein at set copy number
ax = axes[1]
plot_df = sim_data[(sim_data['param']=='k_mRNA_bind') & (sim_data['copy_num'].isin([25,75]))].copy()
sns.lineplot(data=plot_df, x='param_val_norm', y='protein', hue='copy_num', palette={25: reds[1], 75: reds[4]}, ax=ax, 
             legend='full', hue_norm=matplotlib.colors.Normalize(vmin=0, vmax=100,))
plot_df.sort_values('copy_num', inplace=True)
sns.scatterplot(data=plot_df[(plot_df['param_val_norm'].isin(param_val_subset))], x='param_val_norm', y='protein', hue='param_val_norm', palette=greys_subset,
                ax=ax, s=2.25**2*3.14, linewidth=0.5, edgecolor='white', legend=False, zorder=100)
ax.set(yticks=yticks, xlabel='Norm. $k_{mRNA,bind}$', ylim=(0,12e3),
       yticklabels=['']*len(yticks), ylabel='')
sns.move_legend(ax, labels=[25,75], loc='upper right', bbox_to_anchor=(1,1), frameon=False,
                fontsize=base.font_sizes['smaller_size'], handlelength=1, labelspacing=0.3, title='copy number',
                title_fontsize=base.font_sizes['smaller_size'])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['E2']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.45, top=0.4, right=0.3)
axes = subfig.subplots(1,2, gridspec_kw=dict(width_ratios=(1,1), wspace=0.5))

# data: miRE-FF4 vs miRE-FF5 stats
ax = axes[0]
miR_order = ['miRE.FF4','miRE.FF5']
plot_df = stats[((stats['group']=='controller') & (stats['design']==1) & (stats['miR'].isin(miR_order)) & (stats['ts'].isin(['FF4x1','FF5x1','FF6x1']))) &
                (stats['promoter']=='EF1a')].copy()
plot_df['miR'] = plot_df['miR'].astype(pd.api.types.CategoricalDtype(categories=miR_order, ordered=True))
xlim = (-0.5, len(plot_df['miR'].unique())-0.5)
display(plot_df['construct'].unique())

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='miR', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Output mean', xlim=xlim, xlabel='', ylabel='', yscale='log', ylim=(3e1,2.5e4))
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()
ax.set_xticklabels([l.get_text().replace('.','-\n') for l in ax.get_xticklabels()])

# data: vary ts num
ax = axes[1]
plot_df = stats[(stats['group']=='controller') & (stats['miR']=='miRE.FF4') & (stats['design']==1) & (stats['promoter']=='EF1a')].copy()
xlim = (-0.5, len(plot_df['ts_num'].unique())-0.5)
display(plot_df['construct'].unique())

for construct, group in plot_df.groupby('construct'):
    sns.pointplot(data=group, x='ts_num', y='output_gmean', hue='construct', palette=main_palette,
                  ax=ax, marker=main_markers[construct], **scatter_kwargs)
ax.set(title='Output mean', xlim=xlim, xlabel='# target sites', ylabel='', yscale='log', ylim=(3e1,2.5e4),)
marker_baseline = stats.loc[(stats['group']=='marker'), 'output_gmean'].mean()
ax.axhline(marker_baseline, color='black', ls=':')
ax.minorticks_off()

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['G']
rd.plot.adjust_subplot_margins_inches(subfig, left=0.6, bottom=0.3, top=0.4, right=1.22)
axes = subfig.subplots(1,1)

# model: alpha_p sweep
ax = axes
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='α_p']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,12e3),
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')
ax.set_title('Free mRNA\ntranslation, $α_{p}$', color=base.colors['orange'])

fig.savefig(rd.outfile(output_path))

In [ ]:
subfig = subfigures['H']
rd.plot.adjust_subplot_margins_inches(subfig, left=1.4, bottom=0.3, top=0.4, right=1.74)
axes = subfig.subplots(1,1,)

# model: k_miRNA_deg sweep
ax = axes
yticks = np.linspace(0,1e4,5)
plot_df = sim_data[sim_data['param']=='k_miRNA_deg']
sns.lineplot(data=plot_df, x='copy_num', y='protein', hue='param_val', palette=greys, 
             ax=ax, hue_norm=matplotlib.colors.LogNorm(), legend=False)#legend='full')
sns.lineplot(data=plot_df[plot_df['param_val_norm']==1], x='copy_num', y='protein', color='black', ax=ax, linewidth=1.5,
             errorbar=None,)
ax.set(xticks=np.linspace(0,100,5), yticks=yticks, xlabel='copy number', ylim=(0,6e3),
       title='Bound microRNA\ndegradation, $k_{miRNA,deg}$',
       yticklabels=[f'{float(l)/1000:.1f}k' for l in yticks], ylabel='protein (#)')

fig.savefig(rd.outfile(output_path))

In [ ]:
# Save to OneDrive
fig.savefig(rd.outfile(rd.datadir/'manuscripts'/'2024_miR-iFFL'/'figures'/'links'/'fig_model.pdf'))